<a href="https://colab.research.google.com/github/imdeepmind/UdacitySelfDrivingCar/blob/master/Base_Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Getting the into colab
!git clone https://github.com/imdeepmind/UdacitySelfDrivingCar.git
!mkdir data
!mv UdacitySelfDrivingCar/* ./
!rm -r UdacitySelfDrivingCar

Cloning into 'UdacitySelfDrivingCar'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 5242 (delta 24), reused 10 (delta 5), pack-reused 5200
Receiving objects: 100% (5242/5242), 66.04 MiB | 37.78 MiB/s, done.
Resolving deltas: 100% (25/25), done.
mkdir: cannot create directory ‘data’: File exists
mv: cannot move 'UdacitySelfDrivingCar/data' to './data': Directory not empty


In [2]:
!pip install neuralpy-torch

In [16]:
# Dependencies
import pandas as pd
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.transforms as transforms

from neuralpy.models import Sequential
from neuralpy.layers import Conv2D, Dense, Flatten, MaxPool2D
from neuralpy.regularizers import Dropout2D
from neuralpy.activation_functions import ReLU

In [17]:
# Constants
CSV_FILE = './data/driver_log_cleaned.csv'
BATCH_SIZE=32
EPOCH=3

In [18]:
# Reading CSV file
data = pd.read_csv(CSV_FILE)
data.head()

,center_camera,left_camera,right_camera,steering,throttle,reverse,speed
0,./data/IMG/center_2020_09_05_16_40_17_113.jpg,./data/IMG/left_2020_09_05_16_40_17_113.jpg,./data/IMG/right_2020_09_05_16_40_17_113.jpg,0.0,0.0,0,0.000077
1,./data/IMG/center_2020_09_05_16_40_17_214.jpg,./data/IMG/left_2020_09_05_16_40_17_214.jpg,./data/IMG/right_2020_09_05_16_40_17_214.jpg,0.0,0.0,0,0.000077
2,./data/IMG/center_2020_09_05_16_40_17_320.jpg,./data/IMG/left_2020_09_05_16_40_17_320.jpg,./data/IMG/right_2020_09_05_16_40_17_320.jpg,0.0,0.0,0,0.000077
3,./data/IMG/center_2020_09_05_16_40_17_425.jpg,./data/IMG/left_2020_09_05_16_40_17_425.jpg,./data/IMG/right_2020_09_05_16_40_17_425.jpg,0.0,0.0,0,0.000077
4,./data/IMG/center_2020_09_05_16_40_17_531.jpg,./data/IMG/left_2020_09_05_16_40_17_531.jpg,./data/IMG/right_2020_09_05_16_40_17_531.jpg,0.0,0.0,0,0.000077


In [19]:
left_image = data["left_camera"].values
center_image = data["center_camera"].values
right_image = data["right_camera"].values

steering = data["steering"].values

In [20]:
# Custom Data Loader class
class DriveDataset(Dataset):
  def __init__(self, left_images, center_images, right_images, steering, transform=None):
    self.__left_images = left_images
    self.__center_images = center_images
    self.__right_images = right_images

    self.__steering = steering

    self.__transform = transform
  
  def __len__(self):
    return len(self.__steering)
  
  def __getitem__(self, index):
    left = cv2.imread(self.__left_images[index], 1)
    center = cv2.imread(self.__center_images[index], 1)
    right = cv2.imread(self.__right_images[index], 1)

    stacked = np.hstack([left, center, right])

    if self.__transform:
      stacked = self.__transform(stacked)

    steering = torch.tensor(float(self.__steering[index]))

    return stacked, steering

In [21]:
# Making the data loader generator
dataset = DriveDataset(left_image, center_image, right_image, steering, transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(left_image) * .8), int(len(steering) - (len(steering) * .8) + 1) ])

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

In [22]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, input_shape=(3, 160,960)))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=64, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=128, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=128, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())

model.add(Conv2D(filters=256, kernel_size=3))
model.add(ReLU())
model.add(MaxPool2D(kernel_size=2))
model.add(Dropout2D())


model.add(Flatten())

model.add(Dense(1024))
model.add(ReLU())

model.add(Dense(1024))
model.add(ReLU())

model.add(Dense(1))

In [23]:
model.build()
model.summary()

The model is running on cuda:0
Sequential(
  (conv2d_layer_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_2): ReLU()
  (maxpool2d_layer_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_4): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_6): ReLU()
  (maxpool2d_layer_7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_8): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_10): ReLU()
  (maxpool2d_layer_11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_12): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_14): ReLU()
  (maxpool2d_layer_15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d

In [31]:
pytorch_model = model.get_model()
pytorch_model

Sequential(
  (conv2d_layer_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_2): ReLU()
  (maxpool2d_layer_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_4): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_6): ReLU()
  (maxpool2d_layer_7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_8): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_10): ReLU()
  (maxpool2d_layer_11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_12): Dropout2d(p=0.5, inplace=False)
  (conv2d_layer_13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_layer_14): ReLU()
  (maxpool2d_layer_15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2d_layer_16): Dropout2d(p=0.5, in

In [32]:
# Pytorch devoce
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [33]:
# Optimizer and loss function
criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)

In [34]:
for epoch in range(EPOCH):
  dataiter = iter(train_loader)

  for i, data in enumerate(dataiter):
    # unpacking the data
    images, labels = data
    
    # moving the data to device
    images = images.to(device)
    labels = labels.to(device)
    # passing the image through the model and predict something
    out = pytorch_model(images)

    # calculating the loss based on the prediction
    loss = criterion(out, labels.view(-1, 1))
    
    # zero gradient
    optimizer.zero_grad()

    # backpropagation
    loss.backward()
    
    # update the parameters
    optimizer.step()

    # printing a message
    if (i+1) % 10 == 0:
      print(f"Epoch: {epoch+1}:{EPOCH}  Batch: {i+1}  Loss: {loss.item()*10000}")

Epoch: 1:3  Batch: 10  Loss: 329.09877598285675
Epoch: 1:3  Batch: 20  Loss: 231.84694349765778
Epoch: 1:3  Batch: 30  Loss: 68.95240861922503
Epoch: 1:3  Batch: 40  Loss: 133.60518030822277
Epoch: 2:3  Batch: 10  Loss: 287.98747807741165
Epoch: 2:3  Batch: 20  Loss: 295.8093024790287
Epoch: 2:3  Batch: 30  Loss: 143.65453273057938
Epoch: 2:3  Batch: 40  Loss: 282.3583781719208
Epoch: 3:3  Batch: 10  Loss: 87.75011636316776
Epoch: 3:3  Batch: 20  Loss: 274.0849182009697
Epoch: 3:3  Batch: 30  Loss: 412.51830756664276
Epoch: 3:3  Batch: 40  Loss: 149.26616102457047
